In [1]:
import pandas as pd
from tqdm import tqdm

 * raw data 불러오기

In [2]:
# 1. 온라인 행동 데이터
raw_data1_Online_Act = pd.read_csv('./data/data1_Online_Act.csv')

# 2. 거래 정보 데이터
raw_data2_Trade_Inform = pd.read_csv('./data/data2_Trade_Inform.csv')

# 3. 사용자 demo 정보(나이, 성별)
raw_data3_Demog = pd.read_csv('./data/data3_Demog.csv')

# 4. 상품 대중소 분류 정보
raw_data4_Item_Class = pd.read_csv('./data/data4_Item_Class.csv')

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


 * 온라인 데이터 + 외부데이터 불러오기

In [3]:
online_add_exData = pd.read_csv("online_04-6.csv")

In [38]:
online_add_exData.head(3)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,cum_act_7,day,holiday,hour,sech_clac_nm2,prefer_dvc_trfc,hum,temp,pty,r06
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,0,2,0,16,NaN,mobile_app_DIRECT,67.0,22.1,0.0,0.0
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,0,2,0,16,NaN,mobile_app_PUSH,67.0,22.1,0.0,0.0
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,0,2,0,16,NaN,mobile_app_DIRECT,67.0,22.1,0.0,0.0


In [39]:
online_add_exData.shape

(3196362, 31)

 * 온라인 데이터 + 키워드 크롤링을 통해 생성한 중분류 데이터 불러오기

In [5]:
merge_table_add_nm2_all = pd.read_csv("merge_all_create_sech_clac_nm2.csv")
merge_table_add_nm2_all.drop("Unnamed: 0", axis=1, inplace=True)

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [40]:
merge_table_add_nm2_all.head(3)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1,sech_clac_nm2
0,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,...,unknown,NaN,NaN,NaN,NaN,과일선물세트,식품,신선식품,Chilled Foods,Chilled Instant Foods
1,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,unknown,NaN,NaN,NaN,NaN,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services,Living Services


In [41]:
merge_table_add_nm2_all.shape

(3196362, 32)

## (온라인 데이터 + 외부데이터)에 키워드와 중분류 데이터 merge

 * 데이터 정렬
     * clnt_id -> sess_id -> sess_dt(일자) -> hit_tm(시분) -> hit_pss_tm(밀리초) 순으로 정렬(시간순 정렬)

In [11]:
online_add_exData.sort_values(by = ['clnt_id', 'sess_id', 'hit_seq', 'sess_dt', 'hit_tm', 'hit_pss_tm'], axis=0, inplace = True)
merge_table_add_nm2_all.sort_values(by = ['clnt_id', 'sess_id', 'hit_seq', 'sess_dt', 'hit_tm', 'hit_pss_tm'], axis=0, inplace = True)
online_add_exData.reset_index(drop=True, inplace = True)
merge_table_add_nm2_all.reset_index(drop=True, inplace = True)

In [12]:
online_add_exData.head(5)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,cum_act_7,day,holiday,hour,sech_clac_nm2,prefer_dvc_trfc,hum,temp,pty,r06
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,0,2,0,16,NaN,mobile_app_DIRECT,67.0,22.100000,0.0,0.0
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,0,2,0,16,NaN,mobile_app_PUSH,67.0,22.100000,0.0,0.0
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,0,2,0,16,NaN,mobile_app_DIRECT,67.0,22.100000,0.0,0.0
3,1,2,1,0,A01,2019-09-22,14:09,41584,NaN,초등가을잠바,...,0,6,1,14,Jewelry,mobile_app_DIRECT,58.0,18.200001,0.0,0.0
4,1,2,2,0,A01,2019-09-22,14:10,56113,NaN,초등가을점퍼,...,0,6,1,14,Women's Shoes,mobile_app_DIRECT,58.0,18.200001,0.0,0.0


In [15]:
online_add_exData.shape

(3196362, 31)

In [13]:
merge_table_add_nm2_all.head(5)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1,sech_clac_nm2
0,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,...,unknown,NaN,NaN,NaN,NaN,과일선물세트,식품,신선식품,Chilled Foods,Chilled Instant Foods
1,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,unknown,NaN,NaN,NaN,NaN,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services,Living Services
3,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
merge_table_add_nm2_all.shape

(3196362, 32)

 ### merge 할 데이터만 뽑기
     * act_kwd : (검색과 동시간에 수행된)온라인 행동에 대한 검색 키워드
     * sech_kwd_over_3 : 3번 이상 검색된 키워드
     * sech_kwd_cat_0 : 롯데홈쇼핑 사이트에서 sech_kwd_over_3를 검색하여 크롤링한 큰카테고리명
     * sech_kwd_cat_1 : 롯데홈쇼핑 사이트에서 sech_kwd_over_3를 검색하여 크롤링한 작은카테고리명
     * sech_clac_nm1 : sech_kwd_cat_1과 의미상 매칭되는 clac_nm1(상품대분류명)
     * sech_clac_nm2 : sech_clac_nm1를 통하여 대분류 내 중분류 비율에 맞게 생성한 clac_nm2(중분류명)

In [17]:
add_cols = merge_table_add_nm2_all[['act_kwd', 'sech_kwd_over_3', 'sech_kwd_cat_0', 'sech_kwd_cat_1', 'sech_clac_nm1', 'sech_clac_nm2']]
add_cols.head(3)

,act_kwd,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1,sech_clac_nm2
0,과일선물세트,과일선물세트,식품,신선식품,Chilled Foods,Chilled Instant Foods
1,과일선물세트 백화점,NaN,NaN,NaN,NaN,NaN
2,과일바구니,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services,Living Services


In [18]:
add_cols.shape

(3196362, 6)

 * 위에 2개 병합

In [19]:
online_add_exData_kwdData = pd.concat([online_add_exData, add_cols], axis=1)

In [20]:
online_add_exData_kwdData.head(3)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,hum,temp,pty,r06,act_kwd,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1,sech_clac_nm2
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,67.0,22.1,0.0,0.0,과일선물세트,과일선물세트,식품,신선식품,Chilled Foods,Chilled Instant Foods
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,67.0,22.1,0.0,0.0,과일선물세트 백화점,NaN,NaN,NaN,NaN,NaN
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,67.0,22.1,0.0,0.0,과일바구니,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services,Living Services


In [21]:
online_add_exData_kwdData.shape

(3196362, 37)

### 2. 거래정보, 4. 상품명 정보 병합

    * merge의 key역할을 수행하기 위해 raw_data4_Item_Class['pd_c'] data를 raw_data2와 동일한 str 형으로 변경
    
     ### 4. 데이터 병합을 위하여 상품 이름정보의 pd_c 데이터 타입 변경 (int -> str)
     * raw_data2_Trade_Inform에 있는 pd_c열의 경우, NaN과 unknown 값을 포함하기 때문에,  
       자료형을 int나 float로 변경할 수 없음(변경 시, 에러 발생)
     * 따라서 4. 상품 이름 정보의 pd_c를 str 형으로 자료형 변경
     * 그냥 object 형으로만 변경 시, 값이 같아도
         merge_raw123 에서는 pd_c = '0062'(str형)으로 인식하고,
         4. 상품이름 정보에서는 pd_c = 62(int형으로 인식해서 병합 시, 데이터 matching이 안됨

In [22]:
# 4. 상품 이름 데이터의 'pd_c' 열의 값을 4자리 숫자로 변환합니다.
# 예시) '1' => '0001', '62' => '0062', '773' => '0773' 

raw_data4_Item_Class['pd_c'] = raw_data4_Item_Class['pd_c'].astype('str')

cnt_add_zero = 0

for i in range(999):
    cnt_add_zero = 4 - len(raw_data4_Item_Class['pd_c'][i])
    add_zero = "0" * cnt_add_zero
    raw_data4_Item_Class['pd_c'][i] = add_zero + raw_data4_Item_Class['pd_c'][i]

In [25]:
print('key1 sample :', raw_data2_Trade_Inform['pd_c'][0] )
print('key2 sample :',raw_data4_Item_Class['pd_c'][0]  )
print('key1 type :', type(raw_data2_Trade_Inform['pd_c'][0]) )
print('key2 type :', type(raw_data4_Item_Class['pd_c'][0]) )
print('key1 len :', len(raw_data2_Trade_Inform['pd_c']) )
print('key2 len :', len(raw_data4_Item_Class['pd_c']) )

key1 sample : unknown
key2 sample : 0001
key1 type : <class 'str'>
key2 type : <class 'str'>
key1 len : 599961
key2 len : 1667


In [27]:
merge_raw24 = pd.merge(raw_data2_Trade_Inform, raw_data4_Item_Class, on='pd_c', how='left')

In [28]:
merge_raw24

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,clac_nm1,clac_nm2,clac_nm3
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1,NaN,NaN,NaN
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1,NaN,NaN,NaN
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1,NaN,NaN,NaN
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1,NaN,NaN,NaN
4,39423,105124,13,A03,0565,20190920,17:26,2990,1,Fruits,Imported Fruits,Bananas
...,...,...,...,...,...,...,...,...,...,...,...,...
599956,35311,2839,3,B03,0339,20190927,14:09,3500,1,Cosmetics / Beauty Care,Skin Care,Facial Masks
599957,35311,2839,4,B03,0339,20190927,14:09,3600,2,Cosmetics / Beauty Care,Skin Care,Facial Masks
599958,35311,2839,5,B03,0339,20190927,14:09,3500,1,Cosmetics / Beauty Care,Skin Care,Facial Masks
599959,35311,2839,2,B03,0339,20190927,14:09,4500,1,Cosmetics / Beauty Care,Skin Care,Facial Masks


### (1.온라인행동+외부데이터+키워드/중분류) + (2. 거래정보 + 4. 상품명정보 병합)

 * 혹시 있을 꼬임을 위해 key를 제외한 모든 중복 columns 제거

In [32]:
online_add_exData_kwdData.columns

Index(['clnt_id', 'sess_id', 'hit_seq', 'action_type', 'biz_unit', 'sess_dt',
       'hit_tm', 'hit_pss_tm', 'trans_id', 'sech_kwd', 'tot_pag_view_ct',
       'tot_sess_hr_v', 'trfc_src', 'dvc_ctg_nm', 'cum_act_0', 'cum_act_1',
       'cum_act_2', 'cum_act_3', 'cum_act_4', 'cum_act_5', 'cum_act_6',
       'cum_act_7', 'day', 'holiday', 'hour', 'sech_clac_nm2',
       'prefer_dvc_trfc', 'hum', 'temp', 'pty', 'r06', 'act_kwd',
       'sech_kwd_over_3', 'sech_kwd_cat_0', 'sech_kwd_cat_1', 'sech_clac_nm1',
       'sech_clac_nm2'],
      dtype='object')

In [33]:
merge_raw24.columns

Index(['clnt_id', 'trans_id', 'trans_seq', 'biz_unit', 'pd_c', 'de_dt',
       'de_tm', 'buy_am', 'buy_ct', 'clac_nm1', 'clac_nm2', 'clac_nm3'],
      dtype='object')

In [36]:
merge_raw24.drop(['clnt_id', 'biz_unit'], axis=1, inplace = True)

In [37]:
merge_raw24.columns

Index(['trans_id', 'trans_seq', 'pd_c', 'de_dt', 'de_tm', 'buy_am', 'buy_ct',
       'clac_nm1', 'clac_nm2', 'clac_nm3'],
      dtype='object')

In [42]:
print('key1 sample :', online_add_exData_kwdData['trans_id'][0])
print('key2 sample :',merge_raw24['trans_id'][0])
print('key1 type :', type(online_add_exData_kwdData['trans_id'][0]))
print('key2 type :', type(merge_raw24['trans_id'][0]))
print('key1 len :', len(online_add_exData_kwdData['trans_id']))
print('key2 len :', len(merge_raw24['trans_id']))

key1 sample : nan
key2 sample : 104999
key1 type : <class 'numpy.float64'>
key2 type : <class 'numpy.int64'>
key1 len : 3196362
key2 len : 599961


 * key(trans_id) 자료type 맞춰주기

In [50]:
merge_raw24['trans_id'] = merge_raw24['trans_id'].astype('float')

 * 병합

In [52]:
merge_124 = pd.merge(online_add_exData_kwdData, merge_raw24, on = 'trans_id', how = 'left')

In [53]:
merge_124.head(3)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,sech_clac_nm2,trans_seq,pd_c,de_dt,de_tm,buy_am,buy_ct,clac_nm1,clac_nm2,clac_nm3
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,Chilled Instant Foods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,Living Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
merge_124.shape

(3299190, 46)

### + 3.사용자 정보 병합

In [56]:
merge_124.columns

Index(['clnt_id', 'sess_id', 'hit_seq', 'action_type', 'biz_unit', 'sess_dt',
       'hit_tm', 'hit_pss_tm', 'trans_id', 'sech_kwd', 'tot_pag_view_ct',
       'tot_sess_hr_v', 'trfc_src', 'dvc_ctg_nm', 'cum_act_0', 'cum_act_1',
       'cum_act_2', 'cum_act_3', 'cum_act_4', 'cum_act_5', 'cum_act_6',
       'cum_act_7', 'day', 'holiday', 'hour', 'sech_clac_nm2',
       'prefer_dvc_trfc', 'hum', 'temp', 'pty', 'r06', 'act_kwd',
       'sech_kwd_over_3', 'sech_kwd_cat_0', 'sech_kwd_cat_1', 'sech_clac_nm1',
       'sech_clac_nm2', 'trans_seq', 'pd_c', 'de_dt', 'de_tm', 'buy_am',
       'buy_ct', 'clac_nm1', 'clac_nm2', 'clac_nm3'],
      dtype='object')

In [55]:
raw_data3_Demog.columns

Index(['clnt_id', 'clnt_gender', 'clnt_age'], dtype='object')

In [57]:
merge_124['clnt_id'].dtypes

dtype('int64')

In [58]:
raw_data3_Demog['clnt_id'].dtypes

dtype('int64')

In [59]:
merge_all = pd.merge(merge_124, raw_data3_Demog, on = 'clnt_id', how = 'left')

In [60]:
merge_all.head(3)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,pd_c,de_dt,de_tm,buy_am,buy_ct,clac_nm1,clac_nm2,clac_nm3,clnt_gender,clnt_age
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown


In [61]:
merge_all.shape

(3299190, 48)

 * 병합된 데이터 저장(sech_clac_nm2 하나 지우고 다시 저장)

In [62]:
# merge_all.to_csv("Merge_all_data.csv", index=False)

 * 저장된 데이터 재확인

In [63]:
merge_all = pd.read_csv("Merge_all_data.csv")

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [65]:
merge_all.shape

(3299190, 48)

In [64]:
merge_all.columns

Index(['clnt_id', 'sess_id', 'hit_seq', 'action_type', 'biz_unit', 'sess_dt',
       'hit_tm', 'hit_pss_tm', 'trans_id', 'sech_kwd', 'tot_pag_view_ct',
       'tot_sess_hr_v', 'trfc_src', 'dvc_ctg_nm', 'cum_act_0', 'cum_act_1',
       'cum_act_2', 'cum_act_3', 'cum_act_4', 'cum_act_5', 'cum_act_6',
       'cum_act_7', 'day', 'holiday', 'hour', 'sech_clac_nm2',
       'prefer_dvc_trfc', 'hum', 'temp', 'pty', 'r06', 'act_kwd',
       'sech_kwd_over_3', 'sech_kwd_cat_0', 'sech_kwd_cat_1', 'sech_clac_nm1',
       'sech_clac_nm2.1', 'trans_seq', 'pd_c', 'de_dt', 'de_tm', 'buy_am',
       'buy_ct', 'clac_nm1', 'clac_nm2', 'clac_nm3', 'clnt_gender',
       'clnt_age'],
      dtype='object')

 * 중복열 sech_calc_nm2 삭제

In [67]:
merge_all.drop('sech_clac_nm2', axis = 1, inplace = True)

In [68]:
merge_all.rename(columns = {'sech_clac_nm2.1' : 'sech_clac_nm2'}, inplace = True)

 * 병합된 데이터 저장

In [79]:
merge_all.to_csv("Merge_all_data.csv", index=False)

------------------------------------------------------------------------------------------------------------------------------